In [1]:
import sys
sys.path.append('..')

In [2]:
import numpy as np
import os
import tensorflow as tf
import pandas as pd
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import initializers
from preprocessing import parse_aug_fn, parse_fn

In [3]:
train_split, valid_split = ['train[:90%]', 'train[90%:]']
train_data, info = tfds.load('cifar10', split=train_split, with_info=True)
vaild_data = tfds.load('cifar10', split=valid_split)
test_data = tfds.load('cifar10', split='test')

print(train_data)


<PrefetchDataset shapes: {id: (), image: (32, 32, 3), label: ()}, types: {id: tf.string, image: tf.uint8, label: tf.int64}>


In [4]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
batch_size = 64
train_num = int(info.splits['train'].num_examples / 10 )* 9

train_data = train_data.shuffle(train_num)
train_data = train_data.map(map_func=parse_aug_fn, num_parallel_calls=AUTOTUNE)
train_data = train_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

vaild_data = vaild_data.map(map_func=parse_fn, num_parallel_calls=AUTOTUNE)
vaild_data = vaild_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

test_data = test_data.map(map_func=parse_fn, num_parallel_calls=AUTOTUNE)
test_data = test_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)



In [7]:
def bulid_and_train_model(run_name,init):
    inputs = keras.Input(shape=(32, 32, 3))
    x = layers.Conv2D(64, (3, 3), kernel_initializer=init, activation='relu')(inputs)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(128, (3, 3), kernel_initializer=init, activation='relu')(x)
    x = layers.Conv2D(256, (3, 3), kernel_initializer=init, activation='relu')(x)
    x = layers.Conv2D(128, (3, 3), kernel_initializer=init, activation='relu')(x)
    x = layers.Conv2D(64, (3, 3), kernel_initializer=init, activation='relu')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, kernel_initializer=init, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(10, kernel_initializer=init, activation='softmax')(x)
    model = keras.Model(inputs, outputs)
    logfiles = 'lab5-logs/{}'.format(run_name)
    model_mbk = keras.callbacks.TensorBoard(logfiles, histogram_freq=1)
    model_file = logfiles + '/model'.format(run_name)
    model_mckp = keras.callbacks.ModelCheckpoint(model_file, save_best_only=True,model='max',monitor='val_categorical_accuracy')

    model.compile(keras.optimizers.Adam(), loss=keras.losses.CategoricalCrossentropy(), metrics=[keras.metrics.CategoricalAccuracy()])

    model.fit(train_data, epochs=50, validation_data=vaild_data, callbacks=[model_mbk, model_mckp])

In [8]:
session_num = 1
model_dir = 'lab5-logs/models/'
weights_initializers_list = [initializers.RandomNormal(), initializers.GlorotNormal(), initializers.HeNormal()]
for init in weights_initializers_list:
    print('---runing session %d with init %s---' % (session_num, init.__class__.__name__))
    run_name = 'run-%d' % session_num
    bulid_and_train_model(run_name, init)
    session_num += 1


---runing session 1 with init RandomNormal---
Epoch 1/50
  2/704 [..............................] - ETA: 12:39 - loss: 2.3520 - categorical_accuracy: 0.1016WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0254s vs `on_train_batch_end` time: 2.0976s). Check your callbacks.


700/704 [============================>.] - ETA: 0s - loss: 1.9896 - categorical_accuracy: 0.2529INFO:tensorflow:Assets written to: lab5-logs/run-1\model\assets


INFO:tensorflow:Assets written to: lab5-logs/run-1\model\assets


704/704 [==============================] - 19s 27ms/step - loss: 1.9886 - categorical_accuracy: 0.2535 - val_loss: 1.7257 - val_categorical_accuracy: 0.3646
Epoch 2/50
704/704 [==============================] - ETA: 0s - loss: 1.7022 - categorical_accuracy: 0.3776INFO:tensorflow:Assets written to: lab5-logs/run-1\model\assets


INFO:tensorflow:Assets written to: lab5-logs/run-1\model\assets


...

704/704 [==============================] - 18s 26ms/step - loss: 0.7415 - categorical_accuracy: 0.7556 - val_loss: 0.6672 - val_categorical_accuracy: 0.7902
Epoch 42/50
704/704 [==============================] - 9s 13ms/step - loss: 0.7215 - categorical_accuracy: 0.7606 - val_loss: 0.6831 - val_categorical_accuracy: 0.7872
Epoch 43/50
700/704 [============================>.] - ETA: 0s - loss: 0.7166 - categorical_accuracy: 0.7626INFO:tensorflow:Assets written to: lab5-logs/run-3\model\assets


INFO:tensorflow:Assets written to: lab5-logs/run-3\model\assets


704/704 [==============================] - 16s 23ms/step - loss: 0.7164 - categorical_accuracy: 0.7626 - val_loss: 0.6476 - val_categorical_accuracy: 0.7992
Epoch 44/50
704/704 [==============================] - 9s 13ms/step - loss: 0.7070 - categorical_accuracy: 0.7667 - val_loss: 0.7185 - val_categorical_accuracy: 0.7700
Epoch 45/50
704/704 [==============================] - 9s 13ms/step - loss: 0.7019 - categorical_accuracy: 0.7687 - val_loss: 0.7157 - val_categorical_accuracy: 0.7764
Epoch 46/50
704/704 [==============================] - 9s 13ms/step - loss: 0.7043 - categorical_accuracy: 0.7661 - val_loss: 0.6548 - val_categorical_accuracy: 0.7934
Epoch 47/50
704/704 [==============================] - 9s 13ms/step - loss: 0.6969 - categorical_accuracy: 0.7711 - val_loss: 0.6336 - val_categorical_accuracy: 0.7954
Epoch 48/50
704/704 [==============================] - 9s 13ms/step - loss: 0.6906 - categorical_accuracy: 0.7722 - val_loss: 0.7262 - val_categorical_accuracy: 0.7734
Epo

INFO:tensorflow:Assets written to: lab5-logs/run-3\model\assets


704/704 [==============================] - 13s 19ms/step - loss: 0.6849 - categorical_accuracy: 0.7752 - val_loss: 0.6591 - val_categorical_accuracy: 0.8002
Epoch 50/50
704/704 [==============================] - 9s 13ms/step - loss: 0.6864 - categorical_accuracy: 0.7770 - val_loss: 0.6898 - val_categorical_accuracy: 0.7834
